In [ ]:
#Importing H2o library
import h2o

In [ ]:
#Requesting Connection
h2o.init(nthreads = -1, max_mem_size = 8)

In [ ]:
#Loading Ratings file
rating_csv='/Users/pavantej/Desktop/SCIT/sem2/big data/20180701_Batch39_CSE7322c_Recommendation/ml-latest-small/rating_edx.csv

In [ ]:
data = h2o.import_file(rating_csv)

In [ ]:
#loading Movies data

movies_csv='/Users/pavantej/Desktop/SCIT/sem2/big data/20180701_Batch39_CSE7322c_Recommendation/ml-latest-small/movies.csv'
data_movies=h2o.import_file(movies_csv)

In [ ]:
#splitting the data into train and test and validation
splits = data.split_frame(ratios=[0.8, 0.1], seed=1)  

train = splits[0]
valid = splits[1]
test = splits[2]


In [ ]:
#Number of counts
print(train.nrow)
print (valid.nrow)
print (test.nrow)

In [ ]:
y = 'rating'
x = list(data.columns)

In [ ]:

#Building Random Forest 1
from h2o.estimators.random_forest import H2ORandomForestEstimator




In [ ]:
rf_fit1 = H2ORandomForestEstimator(model_id='rf_fit1', seed=1)

In [ ]:
rf_fit1.train(x=x, y=y, training_frame=train)

In [ ]:
rf_perf1 = rf_fit1.model_performance(test)

In [ ]:
#Building Random Forest 2
rf_fit2 = H2ORandomForestEstimator(model_id='rf_fit2', ntrees=500, seed=1)
rf_fit2.train(x=x, y=y, training_frame=train)

In [ ]:
rf_perf2=rf_fit2.model_performance(test)

In [ ]:
#Performance Metrics of both the Random Forest
print (rf_perf1.auc)
print(rf_perf2.auc)

In [ ]:
#Predicting on the test data using the best possible Random forest

In [ ]:
preds_test_rf=rf_fit2.predict(test)

In [ ]:
preds_test_rf

In [ ]:
preds_test_rf.nrows

In [ ]:
test['Preds_RF']=preds_test_rf

In [ ]:
#Gradient Boosting Machine


In [ ]:
#Import H2O GBM:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [ ]:
#Training using a default GBM
# Initialize and train the GBM estimator:

gbm_fit1 = H2OGradientBoostingEstimator(model_id='gbm_fit1', seed=1)
gbm_fit1.train(x=x, y=y, training_frame=train)

In [ ]:
#Training using GBM with more trees
gbm_fit2 = H2OGradientBoostingEstimator(model_id='gbm_fit2', ntrees=500, seed=1)
gbm_fit2.train(x=x, y=y, training_frame=train)

In [ ]:
#Training using GBM with early stopping
# Now let's use early stopping to find optimal ntrees

gbm_fit3 = H2OGradientBoostingEstimator(model_id='gbm_fit3', 
                                        ntrees=500, 
                                        score_tree_interval=5,     #used for early stopping
                                        stopping_rounds=3,         #used for early stopping
                                        #stopping_metric='AUC',     #used for early stopping
                                        stopping_tolerance=0.0005, #used for early stopping
                                        seed=1)

# The use of a validation_frame is recommended with using early stopping
gbm_fit3.train(x=x, y=y, training_frame=train, validation_frame=valid)

In [ ]:
#Compare model performance
gbm_perf1 = gbm_fit1.model_performance(test)
gbm_perf2 = gbm_fit2.model_performance(test)
gbm_perf3 = gbm_fit3.model_performance(test)


In [ ]:
# Retreive test set AUC
print (gbm_perf1.auc)
print (gbm_perf2.auc)
print (gbm_perf3.auc)

In [ ]:
#predicting using the best GBM
preds_GBM=gbm_fit2.predict(test)
preds_GBM

In [ ]:
test['preds_GBM']=preds_GBM
test.show

In [ ]:
#Deep Learning

In [ ]:
# Import H2O DL:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

In [ ]:
# Initialize and train the DL estimator:

dl_fit2 = H2ODeepLearningEstimator(model_id='dl_fit2', 
                                   epochs=20, 
                                   hidden=[5,5], 
                                   stopping_rounds=0,  #disable early stopping
                                   seed=1)
dl_fit2.train(x=x, y=y, training_frame=train)

In [ ]:
dl_fit3 = H2ODeepLearningEstimator(model_id='dl_fit3', 
                                   epochs=20, 
                                   hidden=[10,10],
                                   score_interval=1,          #used for early stopping
                                   stopping_rounds=3,         #used for early stopping
                                   #stopping_metric='AUC',     #used for early stopping
                                   stopping_tolerance=0.0005, #used for early stopping
                                   seed=1)
dl_fit3.train(x=x, y=y, training_frame=train, validation_frame=valid)

In [ ]:
dl_perf2 = dl_fit2.model_performance(test)
dl_perf3 = dl_fit3.model_performance(test)

In [ ]:
print (dl_perf2.auc)
print (dl_perf3.auc)

In [ ]:
preds_NN=dl_fit3.predict(test)
preds_NN

In [ ]:
test['preds_NN']=preds_NN
test

In [ ]:
WA=((0.5*test['Preds_RF'])+(0.3*test['preds_GBM'])+(0.2*test['preds_NN']))
WA

In [ ]:
Avg=(test['Preds_RF']+test['preds_GBM']+test['preds_NN'])/3
Avg

In [ ]:
test['Avg']=Avg